## Se cargan los tres datasets y s ehace un merge para que quede todo en uno solo 

In [33]:
import pandas as pd

## Dataset steam_games

In [34]:
df_steam_games = pd.read_parquet('C:\Proyecto\Proyecto1\DataSets Finales\steam_games.parquet')
df_steam_games['item_id'] = df_steam_games['item_id'].astype(int)
df_steam_games.head(2)

,Unnamed: 0,genres,title,release_date,price,item_id,developer
0,88310,Action,Lost Summoner Kitty,2018.0,4.99,761140,Kotoshiro
1,88310,Casual,Lost Summoner Kitty,2018.0,4.99,761140,Kotoshiro


## Dataset user_review

In [35]:
df_review = pd.read_parquet('C:/Proyecto/Proyecto1/DataSets Finales/user_review.parquet')
df_review['item_id'] = df_review['item_id'].astype(int)
df_review.sample(4)

,Unnamed: 0,user_id,item_id,recommend,sentiment_analysis
12106,4763,76561198116801566,218620,False,2
58600,25427,chicka123,4000,True,0
53219,22668,6935,200900,True,2
22253,8769,76561198118874302,270550,True,2


## Dataset user_items

In [36]:
df_items = pd.read_parquet('C:/Proyecto/Proyecto1/DataSets Finales/users_items.parquet')
df_items.head(3)

,Unnamed: 0,user_id,items_count,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,6.0,0.0
1,1,76561197970982479,277,0.0,0.0
2,2,76561197970982479,277,7.0,0.0


In [37]:
df_items.shape

(2916386, 5)

## Dataset Final

Se hace el merge de los 3 DataFrames a traves de la columna 'item_id'

In [38]:
# se hace merge del dataframe de steam-games y los reviews de usuarios
df_1 = pd.merge(df_steam_games, df_review, on = 'item_id' )

In [39]:
# se hace merge del dataframe anterior con el de los user-items
df = pd.merge(df_1, df_items, on = 'user_id' )

In [40]:
df.shape

(8344484, 15)

Se verifican duplicados y se borran

In [41]:
df.duplicated(subset=df.columns).sum()


0

In [42]:
df.drop_duplicates(keep= 'last', inplace=True)

In [43]:
df.shape

(8344484, 15)

In [44]:
# se eliminan valores nulos
df.dropna(subset= ['playtime_forever','playtime_2weeks'], inplace= True)

In [46]:
df.to_parquet("C:/Proyecto/Proyecto1/DataSets Finales/data_final.parquet")
df.sample()

,Unnamed: 0_x,genres,title,release_date,price,item_id,developer,Unnamed: 0_y,user_id,recommend,sentiment_analysis,Unnamed: 0,items_count,playtime_forever,playtime_2weeks
1619836,90343,Action,Half Minute Hero: The Second Coming,2014.0,9.99,240970,Opus,23233,splitmyurethra,False,0,4914871,85,955.0,0.0


####  Se filtra el dataset con las columnas estrictamnte necesarias final para dar respuesta a cada uno de los endpoints

Dataset para el endpoint 2

In [48]:

df_endpoint_2 = df[['price','user_id','recommend','items_count']] 
df_endpoint_2 = df_endpoint_2.sample(2000000, random_state=42)                  
df_endpoint_2.to_parquet("C:/Proyecto/Proyecto1/DataSets Finales/endpoint_2.parquet")
df_endpoint_2.shape

(2000000, 4)

Dataset enpoint 3

In [49]:
df_endpoint_3 = df[['user_id', 'playtime_forever', 'genres', 'release_date']]
df_endpoint_3 = df_endpoint_3.sample(2000000, random_state=42)
df_endpoint_3.to_parquet("C:/Proyecto/Proyecto1/DataSets Finales/endpoint_3.parquet")
df_endpoint_3.shape

(2000000, 4)

Dataset endpoint 4 y 5

In [50]:
df_endpoint_4_5 = df[['developer', 'recommend', 'release_date', 'sentiment_analysis']]
df_endpoint_4_5 = df_endpoint_4_5.sample(2000000, random_state=42)
df_endpoint_4_5.to_parquet('C:/Proyecto/Proyecto1/DataSets Finales/endpoint_4_5.parquet')
df_endpoint_4_5.shape

(2000000, 4)

Dataset recomendacion de juegos

In [51]:
df_recomendacion =df[['item_id', 'title', 'genres', 'playtime_2weeks', 'recommend', 'sentiment_analysis', ]]
df_recomendacion.head(1)

,item_id,title,genres,playtime_2weeks,recommend,sentiment_analysis
0,282010,Carmageddon Max Pack,Action,0.0,True,1


In [52]:
# se filtra por "recommend"
df_recomendacion = df_recomendacion[df_recomendacion['recommend']==True]

# se filtra por analisis de sentimientos positivos
df_recomendacion = df_recomendacion[df_recomendacion['sentiment_analysis'] == 2]

# se borra la columna 'recommend' ya que no es necesaria
df_recomendacion = df_recomendacion.drop(['recommend'], axis=1)

In [53]:
"""Sumamos la cantidad de horas jugadas por juego en las últimas dos semanas y la cantidad de recoendaciones positivas, agrupado por juego. Tomamos la cantidad jugada en las últimas dos semanas y lo usamos para el modelo de recomendación, entendiendo que esta cantidad de horas da una tendencia para poder recomendar. No tenemos demasiados parámetros para comparar similitud de coseno, por lo que una tendencia puede ser, en algunos casos, una buena manera de recomendar."""
df_recomendacion = df_recomendacion.groupby(['item_id', 'title', 'genres'], as_index=False).agg({
    'sentiment_analysis': 'count',
    'playtime_2weeks': 'sum',
 })

# Ordenamos de manera descendente en función del análisis de sentimiento.
df_recomendacion = df_recomendacion.sort_values(by='sentiment_analysis', ascending=False)
df_recomendacion = df_recomendacion.groupby(['item_id', 'title']).agg({
    'genres': lambda x: list(x),
    'sentiment_analysis': 'sum',
    'playtime_2weeks': 'sum'
}).reset_index()

In [55]:
# se exporta a la carpeta de datasets para ser consumido por la API
df_recomendacion.to_parquet('C:/Proyecto/Proyecto1/DataSets Finales/recomendacion.parquet')